## 1. Data Load


In [1]:
import os
import sys

if os.getcwd().split('/')[-1] == 'notebooks':
    os.chdir('../')
os.getcwd()

'/home/claude/development/uzh__advanced_topics_in_ai'

In [2]:
import plotly.io as pio
pio.renderers.default = 'jupyterlab+svg'
from speakeasypy import Speakeasy, Chatroom
from typing import List
import time


# NOTE: You might have to download a few things for nltk to work properly
import nltk
from nltk.corpus import wordnet as wn
from nltk import Tree
# nltk.data.path.append('/Users/melihserin/Desktop/ATAI/env/lib/python3.12/site-packages/nltk_data')

# NOTE: You might have to download the en_core_web_sm model for this to work
import spacy
from spacy import displacy
from spacy_wordnet.wordnet_annotator import WordnetAnnotator 
import os

from src.utils import *
from src.part_of_speech import FactualBotPartOfSpeech
from src.ner import FactualBotNER
from src.recommendation import RecommendationBot
from src.simple_recommendation import SimpleRecommendationBot

In [3]:
g = load_graph()
entity_emb, relation_emb = load_embeddings()
ent2id, id2ent, rel2id, id2rel, ent2lbl, lbl2ent = load_dictionaries(g)
WD, WDT, SCHEMA, DDIS, RDFS = load_namespaces()
nodes, predicates = load_nodes_and_preciates(g)
movies = load_movies(g, nodes)
url2nodes = load_url2nodes(nodes, ent2lbl)
genres = load_genres(g, predicates, ent2lbl)
cast_members = load_cast_members(g, ent2lbl)
crowd_data = load_crowd_data()

Graph loaded.
Embeddings loaded.
Dictionaries loaded.
Namespaces loaded.
Nodes loaded.
Predicates loaded.
Namespaces loaded.
Namespaces loaded.
Crowd dataset loaded.


In [4]:
TEST_RECOMMENDATIONS = [
    "I loved Inception. Can you recommend similar movies?",
    "I’m in the mood for something funny. Any good comedies you’d recommend?",
    "Can you suggest some good action movies?",
    "I want to watch comedy movies.",
    "Recommend me a drama movie",
    "Can you recommend me a movie with Ryan Gosling?"
]

In [5]:
TEST_QUESTIONS = [
    "What is the genre of Good Neighbors?",
    'Who directed Apocalypse Now?',
    "Who is the director of Star Wars Episode VI - Return of the Jedi?",
    "Who is the screenwriter of The Masked Gang: Cyprus?",
    'When was The Godfather released?',
    "Who is the producer of Inception?",
    "Who composed the soundtrack for Jurassic Park?",
    "When was Pulp Fiction released?",
    "Who played the lead role in The Matrix?",
    "Who directed Blade Runner 2049?",
    "What is the running time of The Shawshank Redemption?",
    "Who was the cinematographer for Mad Max: Fury Road?",
    "When did Titanic premiere?",
    "Who wrote the screenplay for The Social Network?",
    "What is the box office gross of Avatar?",
    "Who edited the movie Parasite?",
    "What is the budget of Halloween?",
    "Who starred as the main character in Forrest Gump?",
    "When was Interstellar first released?",
    "Who is the production designer of Dune (2021)?",
    "Who is the production designer of Dune?",
]

## Part of Speech - Factual Questions

In [6]:
bot_pos = FactualBotPartOfSpeech(g, nodes, predicates)
bot_pos.get_factual__pos('Who is the director of Forrest Gump?')

'According to the graph, I think the answer is Robert Zemeckis.'

## NER - Factual Questions

In [7]:
bot_ner = FactualBotNER(g, nodes,url2nodes, predicates, entity_emb, relation_emb, ent2id, rel2id, ent2lbl, lbl2ent, id2ent,crowd_data)
bot_ner('Who is the director of Forrest Gump?')

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


'According to the the graph, the director of Forrest Gump is Robert Zemeckis.'

In [8]:
bot_ner("What is the box office of The Princess and the Frog?")

'The box office of The Princess and the Frog is 267000000.\n[Crowd, inter-rater agreement 0.6182, The answer distribution for this specific was 2 support votes, 1 reject votes]'

In [9]:
bot_ner("Who is the executive producer of X-Men: First Class?")

'The executive producer of X-Men: First Class is Sheryl Lee Ralph.\n[Crowd, inter-rater agreement 0.6317, The answer distribution for this specific was 2 support votes, 1 reject votes]'

In [10]:
bot_ner("Can you tell me the publication date of Tom Meets Zizou?")

'The publication date of Tom Meets Zizou is 2010-10-01.\n[Crowd, inter-rater agreement 0.5200, The answer distribution for this specific was 0 support votes, 3 reject votes]'

## Recommendation: NER Approach

In [11]:
del sys.modules['src.recommendation']
from src.recommendation import RecommendationBot

In [12]:
bot_rec = RecommendationBot(bot_ner, g, nodes, predicates, genres, movies, url2nodes, entity_emb, rel2id, ent2id, id2ent, relation_emb,cast_members)
print(bot_rec.get_recommendation("I want to watch comedy movies."))

If you like comedy movies, I would recommend you watch Much Ado About Nothing.


In [13]:
question = "Recommend movies Ryan Gosling and Tom Hanks played in."
bot_rec.get_recommendation(question)

'Oh, Ryan Gosling and others you mentioned have great films! I would recommend you watch The Terminal,The Post, and Bridge of Spies.'

In [14]:
for question in TEST_RECOMMENDATIONS:
    print(f"Question: {question}")
    print('\t', bot_rec.get_recommendation(question))
    print()

Question: I loved Inception. Can you recommend similar movies?
	 Based on what you like, I would recommend you watching movie with the genres drama, thriller, comedy such as Interstellar.

Question: I’m in the mood for something funny. Any good comedies you’d recommend?
	 Sorry, I cannot recommend you a movie based on your query. The reasons might be that I do not know the movies you mentioned or there is a minor problem with the format of your input. You might want to re-check and/or rephrase your sentence. I will be waiting here. 

Question: Can you suggest some good action movies?
	 If you like action movies, I would recommend you watch The Lost Samaritan.

Question: I want to watch comedy movies.
	 If you like comedy movies, I would recommend you watch Much Ado About Nothing.

Question: Recommend me a drama movie
	 If you like drama movies, I would recommend you watch The House of Bernarda Alba.

Question: Can you recommend me a movie with Ryan Gosling?
	 Oh, Ryan Gosling has great

## Recommendation: Vector Similarity Ideas

In [43]:
del sys.modules['src.simple_recommendation']
from src.simple_recommendation import SimpleRecommendationBot

In [44]:
bot_simple_rec = SimpleRecommendationBot(g)

Namespaces loaded.


In [42]:
import random
random.choice(bot_simple_rec.actor_images['nm0331516'])

'0670/rm3168210944'

In [48]:
bot_simple_rec.show_images('show me an image of Mark Hamill')

'image:3301/rm927415296'

In [49]:
bot_simple_rec.get_entities('Given that I like A.I. Artificial Intelligence, can you recommend some movies from leonardo dicaprio and michael sheen?')

(['a.i. artificial intelligence', 'michael'],
 [],
 ['leonardo dicaprio', 'michael sheen'])

In [50]:
test_string = """
    coming-of-age the beauty and the beast with leonardo dicaprio benedict cumberbatch
    What movies can you recommend if I like The, beauty and the beast Life of Pi,
    Lion King, the Lion King, Star Wars, Harry Potter and the Hunger Games?
"""
found_movies, found_genres, found_actors = bot_simple_rec.get_entities(test_string)
print('Found movies: ', found_movies)
print('Found genres: ', found_genres)
print('Found actors: ', found_actors)

Found movies:  ['the lion king', 'life of pi', 'the hunger']
Found genres:  ['coming-of-age story']
Found actors:  ['benedict cumberbatch', 'leonardo dicaprio']


In [54]:
movie2idx = {movie: i for i, movie in enumerate(bot_simple_rec.movie_genres.keys())}

for question in TEST_RECOMMENDATIONS:
    print(question)
    found_movies, found_genres, found_actors = bot_simple_rec.get_entities(question)
    print(f"\tDetected movies: {found_movies}")
    print(f"\tDetected genres: {found_genres}")
    print(f"\tDetected actors: {found_actors}")
    print()
    print(f"Recommendation:\t\t{bot_simple_rec.recommend_movies(question)}")
    print(f"Images:\t\t\t{bot_simple_rec.show_images(question)}")
    print()


I loved Inception. Can you recommend similar movies?
	Detected movies: ['loved']
	Detected genres: []
	Detected actors: []

Recommendation:		Let me think... Ah, yes. Have you tried watching: 'Chaos', 'A True Mob Story' or maybe even 'Shanghai Grand'?
Images:			I didn't find the actor you were looking for. Could you rephrase?

I’m in the mood for something funny. Any good comedies you’d recommend?
	Detected movies: ['good']
	Detected genres: []
	Detected actors: []

Recommendation:		Let me think... Ah, yes. Have you tried watching: 'Linger', 'Blind Mountain' or maybe even 'Cairo Time'?
Images:			I didn't find the actor you were looking for. Could you rephrase?

Can you suggest some good action movies?
	Detected movies: ['good']
	Detected genres: ['action film']
	Detected actors: []

Recommendation:		Let me think... Ah, yes. Have you tried watching: 'Undisputed II: Last Man Standing', 'I Love You Phillip Morris' or maybe even 'Shinjuku Incident'?
Images:			I didn't find the actor you wer

## Evaluation 4

In [55]:
from decouple import config
DEFAULT_HOST_URL = config("UZH_SPEAKEASY_HOST")
listen_freq = 2


class Agent:
    def __init__(self, username, password):
        self.username = username
        # Initialize the Speakeasy Python framework and login.
        self.speakeasy = Speakeasy(host=DEFAULT_HOST_URL, username=username, password=password)
        self.speakeasy.login()  # This framework will help you log out automatically when the program terminates.

    def any_in_list(self, string: str, elements: List[str]) -> bool:
        return any([element in string for element in elements])

    def answer_query(self, query):
        if self.any_in_list(query, ['show', 'image', 'picture']) or (('look' in query) and ('like' in query)):
            # do image recommendation
            answer = bot_simple_rec.show_images(query)
            return answer

        if self.any_in_list(query, ['recommend', 'suggest', 'like']):
            # do movie recommendation
            answer = bot_simple_rec.recommend_movies(query)
            if 'Could you rephrase' in answer:
                answer = bot_rec.get_recommendation(query)

            return answer

        # do fact/embedding/crowd recommendation
        answer = bot_ner(query)
        if 'Sorry, I could not' in answer:
            answer = bot_pos.get_factual__pos(query)

        return answer


    def listen(self):
        while True:
            # only check active chatrooms (i.e., remaining_time > 0) if active=True.
            rooms: List[Chatroom] = self.speakeasy.get_rooms(active=True)
            for room in rooms:
                if not room.initiated:
                    # send a welcome message if room is not initiated
                    room.post_messages(f'Hello! And Gruetzi, Merhaba, Bonjour! How can I help you today?')
                    room.initiated = True
                # Retrieve messages from this chat room.
                # If only_partner=True, it filters out messages sent by the current bot.
                # If only_new=True, it filters out messages that have already been marked as processed.
                for message in room.get_messages(only_partner=True, only_new=True):
                    print(
                        f"\t- Chatroom {room.room_id} "
                        f"- new message #{message.ordinal}: '{message.message}' "
                        f"- {self.get_time()}")

                    # Implement your agent here #
                    if (message.message.lower() == "hi") | (message.message.lower() == "hello"):
                        answer='Hello! And Gruetzi, Merhaba, Bonjour! How can I help you today?'
                    else:
                        try:
                            answer = self.answer_query(message.message)
                        except:
                            answer = "Sorry :( I could not understand you. Can you rephrase your question?"
                    # Send a message to the corresponding chat room using the post_messages method of the room object.
                    room.post_messages(f"{answer.encode('utf-8')}")
                    # Mark the message as processed, so it will be filtered out when retrieving new messages.
                    room.mark_as_processed(message)

                # Retrieve reactions from this chat room.
                # If only_new=True, it filters out reactions that have already been marked as processed.
                for reaction in room.get_reactions(only_new=True):
                    print(
                        f"\t- Chatroom {room.room_id} "
                        f"- new reaction #{reaction.message_ordinal}: '{reaction.type}' "
                        f"- {self.get_time()}")

                    # Implement your agent here #

                    room.post_messages(f"Received your reaction: '{reaction.type}' ")
                    room.mark_as_processed(reaction)

            time.sleep(listen_freq)

    @staticmethod
    def get_time():
        return time.strftime("%H:%M:%S, %d-%m-%Y", time.localtime())


In [ ]:
demo_bot = Agent(config("UZH_BOT_USERNAME"), config("UZH_BOT_PASSWORD"))
demo_bot.listen()

Login successful. Session token: eonOZ49fukaWbJUoxkvXUXjFb7ue7TT_
	- Chatroom c22c971e-a270-421e-9c6b-b5cc7f1cf34b - new message #1: 'I'd like to see an image of Mark Hamill' - 23:17:00, 05-12-2024
	- Chatroom c22c971e-a270-421e-9c6b-b5cc7f1cf34b - new message #3: 'can you show me an image of Leonardo Dicaprio' - 23:17:29, 05-12-2024
	- Chatroom c22c971e-a270-421e-9c6b-b5cc7f1cf34b - new message #5: 'I loved Inception. Can you recommend similar movies?' - 23:18:11, 05-12-2024
	- Chatroom c22c971e-a270-421e-9c6b-b5cc7f1cf34b - new message #7: 'coming-of-age the beauty and the beast with leonardo dicaprio benedict cumberbatch
    What movies can you recommend if I like The, beauty and the beast Life of Pi,
    Lion King, the Lion King, Star Wars, Harry Potter and the Hunger Games?' - 23:18:54, 05-12-2024
	- Chatroom c22c971e-a270-421e-9c6b-b5cc7f1cf34b - new message #9: 'Given that I like A.I. Artificial Intelligence, can you recommend some movies from leonardo dicaprio and michael sheen